Inladen data, converteren naar gpkg (dan kun je met sqlite werken en lagen toevoegen)
shp gehaald van https://maps.rijkswaterstaat.nl/dataregister/srv/dut/catalog.search#/metadata/db60a314-5583-437d-a2ff-1e59cc57704f

In [16]:
import os
import logging
from osgeo import ogr, gdal

class Logger:
    def __init__(self, log_dir="log"):
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        log_file = os.path.join(log_dir, "conversion.log")
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - %(levelname)s - %(message)s",
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger()

    def info(self, message):
        self.logger.info(message)

    def error(self, message):
        self.logger.error(message)

class ShapefileConverter:
    def __init__(self, logger):
        self.logger = logger

    def convert_shapefiles_to_geopackage(self, input_shapefiles, output_geopackage):
        try:
            if not input_shapefiles:
                raise ValueError("No input shapefiles provided.")

            for input_shapefile in input_shapefiles:
                if not os.path.exists(input_shapefile):
                    self.logger.error(f"Input shapefile not found: {input_shapefile}")
                    raise FileNotFoundError(f"Shapefile not found: {input_shapefile}")

                self.logger.info(f"Processing shapefile: {input_shapefile}")

                try:
                    srcDS = gdal.OpenEx(input_shapefile, allowed_drivers=['ESRI Shapefile'])
                    if srcDS is None:
                        raise RuntimeError(f"Failed to open shapefile: {input_shapefile}")

                    options = gdal.VectorTranslateOptions(
                        format='GPKG', accessMode='append', srcSRS='EPSG:28992', dstSRS='EPSG:28992',
                        addFields=True
                    )

                    gdal.VectorTranslate(srcDS=srcDS, destNameOrDestDS=output_geopackage, options=options)
                    self.logger.info(f"Successfully converted: {input_shapefile} to {output_geopackage}")

                    srcDS.Close()

                except Exception as e:
                    self.logger.error(f"Error processing shapefile {input_shapefile}: {e}")
                    raise

        except Exception as e:
            self.logger.error(f"Unexpected error during conversion: {e}")
            raise

        self.logger.info("All shapefiles processed.")
        return 'success'

if __name__ == "__main__":
    # File paths
    input_shapefiles = [
        r"..\data\vaarweg_markering_drijvend_detail\vaarweg_markering_drijvend_detailPoint.shp",
        r"..\data\vaarweg_markering_vast_detail\vaarweg_markering_vast_detailPoint.shp"
    ]
    output_geopackage = r"..\data\converted_datav1.gpkg"

    # Initialize logger and converter
    logger = Logger()
    converter = ShapefileConverter(logger)

    # Convert shapefiles to GeoPackage
    converter.convert_shapefiles_to_geopackage(input_shapefiles, output_geopackage)


2024-12-12 11:05:16,920 - INFO - Processing shapefile: ..\data\vaarweg_markering_drijvend_detail\vaarweg_markering_drijvend_detailPoint.shp
2024-12-12 11:05:17,181 - INFO - Successfully converted: ..\data\vaarweg_markering_drijvend_detail\vaarweg_markering_drijvend_detailPoint.shp to ..\data\converted_datav1.gpkg
2024-12-12 11:05:17,181 - INFO - Processing shapefile: ..\data\vaarweg_markering_vast_detail\vaarweg_markering_vast_detailPoint.shp
2024-12-12 11:05:17,631 - INFO - Successfully converted: ..\data\vaarweg_markering_vast_detail\vaarweg_markering_vast_detailPoint.shp to ..\data\converted_datav1.gpkg
2024-12-12 11:05:17,631 - INFO - All shapefiles processed.


In [ ]:
# Import necessary libraries
import pyarrow as pa
import time
import numpy as np
import geopandas as gp
import pandas as pd
from geopandas.array import from_wkb
from osgeo import ogr

# Enable OGR exceptions for better error handling
ogr.UseExceptions()

# Set pandas display option
pd.set_option('display.max_columns', None)

def load(gpkg_path, layername):
    """
    Loads a specified layer from a GeoPackage and returns it as a GeoDataFrame.

    Parameters:
    - gpkg_path (str): Path to the GeoPackage file.
    - layername (str): Name of the layer to load.

    Returns:
    - gdf (GeoDataFrame): The loaded layer as a GeoDataFrame.
    """
    t1 = time.time()
    datasource = None
    try:
        # Connect to the GeoPackage datasource
        driver = ogr.GetDriverByName('GPKG')
        datasource = driver.Open(gpkg_path, 0)  # 0 = read-only
        if datasource is None:
            raise ValueError(f"Failed to open GeoPackage: {gpkg_path}")

        # Access the specified layer
        layer = datasource.GetLayerByName(layername)
        if layer is None:
            raise ValueError(f"Layer '{layername}' not found in {gpkg_path}")
        logger.info(f"Loaded layer '{layername}' from {gpkg_path}")

        # Create Arrow streams
        arrow_stream = layer.GetArrowStreamAsPyArrow()
        if arrow_stream is None:
            raise RuntimeError("Failed to retrieve Arrow stream from the layer.")
        logger.info("Loaded OGR Arrow stream")

        schema = arrow_stream.schema  # Retrieve schema
        logger.debug(f"Schema retrieved: {schema}")

        # Extract field names from the schema
        fields = [field for field in schema]
        field_names = [field.name for field in fields if field.name]  # Ensure field has a name
        logger.info(f"Field names extracted: {field_names}")

        # Define schema_object using field names
        schema_object = pa.schema([schema.field(i) for i in range(schema.num_fields)])
        logger.debug(f"Schema object defined: {schema_object}")

        # Create table from batches
        batches = []
        for record_batch in arrow_stream:
            # Access fields by name
            arrays = [record_batch.field(field_name) for field_name in field_names]
            batch = pa.RecordBatch.from_arrays(arrays, schema=schema_object)
            batches.append(batch)

        # Combine batches into a single table
        table = pa.Table.from_batches(batches).combine_chunks()
        logger.info(f"Created PyArrow table with {table.num_rows} rows and {table.num_columns} columns.")

        # Convert table to pandas DataFrame
        df = table.to_pandas()
        logger.info("Converted PyArrow table to pandas DataFrame")

        # Identify geometry column (assuming it's named 'geom' or similar)
        # Adjust the geometry column name as per your dataset
        geometry_column = None
        for col in df.columns:
            if 'geom' in col.lower() or 'geometry' in col.lower():
                geometry_column = col
                break

        if geometry_column is None:
            raise ValueError("No geometry column found in the data.")

        # Convert WKB to geometry using GeoPandas
        geometry = from_wkb(df[geometry_column])
        gdf = gp.GeoDataFrame(df, geometry=geometry)
        logger.info("Converted DataFrame to GeoDataFrame")

        t2 = time.time()
        logger.info(f"Data loaded successfully in {t2 - t1:.2f} seconds.")

        return gdf

    except Exception as e:
        logger.error(f"An error occurred while loading the layer '{layername}': {e}")
        raise

    finally:
        if datasource:
            datasource.Close()
            logger.info(f"Closed the GeoPackage datasource: {gpkg_path}")

# Example Usage
if __name__ == "__main__":

    gpkg_path = r'..\data\converted_data.gpkg'  # Update with your actual path
    layer_name = 'vaarweg_markering_drijvend_detailPoint'  # Replace with your actual layer name

    gdf = load(gpkg_path, layer_name)
    print(gdf.head())  # Display first few rows for verification



2024-12-12 10:50:42,203 - INFO - Loaded layer 'vaarweg_markering_drijvend_detailPoint' from ..\data\converted_data.gpkg
2024-12-12 10:50:42,203 - INFO - Loaded OGR Arrow stream
2024-12-12 10:50:42,203 - INFO - Field names extracted: ['fid', 'vaarwater', 'benam_cod', 'benaming', 'inbedrijf', 'x_rd', 'y_rd', 'obj_soort', 'iala_categ', 'n_wgs_gms', 'e_wgs_gms', 'n_wgs_gm', 'e_wgs_gm', 'obj_vorm_c', 'obj_vorm', 'obj_kleur_', 'obj_kleur', 'kleurpatr_', 'kleurpatr', 'v_tt_c', 'tt_toptek', 'tt_kleur_c', 'tt_kleur', 'tt_pat_c', 'tt_klr_pat', 'sign_kar_c', 'sign_kar', 'sign_gr_c', 'sign_groep', 'sign_perio', 'racon_code', 'licht_kl_c', 'licht_klr', 'opgeheven', 'x_wgs84', 'y_wgs84', 'vorm_kleur', 's57_id', 'geom']
2024-12-12 10:50:42,222 - INFO - Created PyArrow table with 9911 rows and 39 columns.
2024-12-12 10:50:42,247 - INFO - Converted PyArrow table to pandas DataFrame
2024-12-12 10:50:42,254 - INFO - Converted DataFrame to GeoDataFrame
2024-12-12 10:50:42,255 - INFO - Data loaded successf

   fid       vaarwater      benam_cod benaming   inbedrijf        x_rd  \
0    1  AARDAPPELENGAT  VW-A    -0101     A 21  01-01-2005  72384,1773   
1    2  AARDAPPELENGAT  VW-A    -0102     A 19  01-01-2005  71915,3833   
2    3  AARDAPPELENGAT  VW-A    -0103     A 17  01-01-2005  71305,1921   
3    4  AARDAPPELENGAT  VW-A    -0104     A 18  01-01-2005  71394,1015   
4    5  AARDAPPELENGAT  VW-A    -0105     A 15  01-01-2005  70730,0620   

          y_rd obj_soort iala_categ    n_wgs_gms     e_wgs_gms    n_wgs_gm  \
0  421540,5036  SK31 630          4  51.46.35.11  004.11.24.30  51.46.5852   
1  421600,9262  SK31 630          4  51.46.36.81  004.10.59.79  51.46.6135   
2  421800,7938  SK31 630          4  51.46.42.95  004.10.27.79  51.46.7158   
3  421958,7968  SK32 630          4  51.46.48.11  004.10.32.29  51.46.8018   
4  421975,8322  SK31 630          4  51.46.48.30  004.09.57.64  51.46.8050   

      e_wgs_gm obj_vorm_c obj_vorm obj_kleur_ obj_kleur kleurpatr_  \
0  004.11.4050  

In [ ]:
#foutgevoelig, met het oog uit pdf gehaald en bekeken in Qgis
#aanwezigheid en locatie boei is in Qgis gecheckt.
#vaarwater : [boeien] relaties, vaarwater benodigd omdat sommige boeiennamen dubbel voorkwamen (bijvoorbeeld K 2 in ijselmeer)
vaarwater_boeien = {"KRAMMER": ["K 2"],
               "KEETEN-MASTGAT-ZIJPE" : ["KT 42-ZG 1", 
                                         "KEETEN A",
                                         "KEETEN B",
                                         "KT 1-EV 2",
                                         "KT 2",
                                         "KT 10",
                                         "KT 13",
                                         "KT 14",
                                         "KT 18", 
                                         "KT 21", 
                                         "KT 24", 
                                         "KT 28", 
                                         "KT 31", 
                                         "KT 27-KR 2"],
               "ENGELSCHE VAARWATER": ["EV 6",
                                       "EV 7-O 4",
                                       "EV 10"
                                       ],
               "OOSTERSCHELDE": ["O 1",
                                 "O 11-Z 2",
                                 "O 15-SAS 2",
                                 "O 18",
                                 "O 22",
                                 "O 25-SVY 2",
                                 "O 28-D 15",
                                 "O 33"],
               "ZANDKREEK": ["Z 1"],       #start 4, niet onderdeel?
               "THOLENSCHE GAT":["TG 4",  #start 3
                                  "TG 3"], #start 3
               "PIETERMANSKREEK": ["PK 4",
                                   "PK 8"],
               "LODIJKSCHE GAT": ["LG 5-PK 2",
                                  "LG 8",
                                  "LG 14"],
               "KRABBENKREEK":["KR 1", "KR 7", "KR 8", "KR 15"]}
#aantal boeien in dict
lenb = 0
for i in vaarwater_boeien:
    lenb += len(vaarwater_boeien[i]) 
print(lenb)#38 features
#boeien selecteren uit gdf
gdf_8uren = gdf[gdf.apply(lambda row: row["benaming"] in vaarwater_boeien.get(row["vaarwater"], []), axis=1)]
#aantal boeien gdf
print(len(gdf_8uren)) #38 features

#checks
if not len(gdf_8uren) == lenb:
    print("oeps, wss typefout in dict")
if not len(gdf_8uren) == 38:
    print("oeps, niet aantal als in kaart.pdf")


38
38
